# Weapons shipment

#### In this notebook we will process a .docx file that contains the purchase and sale of weapons between countries around the world. The period of time currently available is between 1940-2017

#### For all deals, the following information is provided:

    Number ordered            — the number of items ordered under the deal
    Weapon designation        — the designation of the weapon system concerned
    Weapon description        — description of the weapon system concerned
    Year of order/licence     — the year the order was placed or, in the case of licensed production, the licence was issued
    Year(s) of deliveries     — the year or years during which deliveries took place. 
                                If no deliveries have yet been made, this field is left blank.
    Number delivered/produced — the number of items delivered or produced under the deal
    Comments                  — any additional information that is known about the deal.
    
- To begin, run in the anaconda console: <br>
  - pip install python-docx

- If necessary, run in the anaconda console: <br>
  - pip install package-name

In [1]:
import pandas as pd
import numpy as np
from docx import Document
import math
import warnings
warnings.filterwarnings("ignore")

- Load the .docx file and structure rows and columns. <br>
   We also match the formats of some cells (null, nan, spaces, tabs ...)

In [2]:
cols = ['country', 'number_ordered', 'weapon_designation', 'weapon_description', 
        'year_of_order', 'year_of_deliveries', 'number_delivered', 'comments', 'comments2']

# open .docx
f = open('05_Input_Trade-Register-1950-2017.docx', 'rb')
document = Document(f)
f.close()

# Extract text
text = []
for paragraph in document.paragraphs:
        text.append(paragraph.text)
        
# Split by column (\t)
test = [n.split('\t') for n in text]
df   = pd.DataFrame(test, columns = cols)

# There are several types of white space. we unify them
df.replace('     ', ''    , inplace=True)
df.replace(''     , np.nan, inplace=True)
df.replace('. .'  , ''    , inplace = True)
df.fillna(value=pd.np.nan , inplace=True)

# delete empty rows
df = df.dropna(how = 'all')
df = df.reset_index(drop=True)

# In the country column we have the suppliers and those who receive weapons.
# We have to separate it. For this we add 2 columns
# Suppliers start in the rows whose only fill value is country
df['country_supplier']  = np.nan
df['country_recipient'] = np.nan

# To better understand the data order columns
cols_new = ['country', 'country_supplier','country_recipient', 'number_ordered', 'weapon_designation', 'weapon_description',
            'year_of_order', 'year_of_deliveries', 'number_delivered', 'comments','comments2']

df = df[cols_new]

# Count how many non-zero cells we have per rows. It will help us with the filters
df['count_values'] = df.apply(lambda x: x.count(), axis = 1)

# If the row only has one value, the country is a supplier
# If the row has between 5-8 values, the country is buyer
df['country_supplier']  = df.loc[df['count_values'] == 1,            ['country']]
df['country_recipient'] = df.loc[df['count_values'].isin([5,6,7,8]), ['country']]
df = df.drop(['country'], axis = 1)

# Fix blanks and characters
df['country_recipient'] = df['country_recipient'].str.replace('R: ', '')
df['country_recipient'] = df['country_recipient'].str.replace('     ', '')
df.replace('', np.nan, inplace=True)

# Show df
df.head()

,country_supplier,country_recipient,number_ordered,weapon_designation,weapon_description,year_of_order,year_of_deliveries,number_delivered,comments,comments2,count_values
0,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NaN,Burkina Faso,(12),M-43 120mm,Mortar,(2011),2011,12,Probably second-hand,NaN,8
2,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,NaN,Angola,4,N-262 Fregate,Transport aircraft,1980,1980,4,Second-hand; incl for MP,NaN,8
4,NaN,Nigeria,(2),Il-28,Bomber aircraft,1967,1968,(2),Second-hand,NaN,8


- Fill cells that are empty for supplier countries and buyer countries

In [3]:
# For the country supplier, if the cell is blank, take the value of cell -1
for i in range(1, len(df)):
    if pd.isnull(df.iloc[i,0]):
        df.loc[i, 'country_supplier'] = df.loc[i-1, 'country_supplier']
        
# Repeat for the buyer countries
for i in range(1, len(df)):
    if pd.isnull(df.iloc[i,1]):
        df.loc[i, 'country_recipient'] = df.loc[i-1, 'country_recipient']
        
# Delete the rows that no longer provide information
df2 = df[df['count_values'] != 1]
df2 = df2.reset_index(drop=True)

# Show df2
df2.head()

,country_supplier,country_recipient,number_ordered,weapon_designation,weapon_description,year_of_order,year_of_deliveries,number_delivered,comments,comments2,count_values
0,Albania,Burkina Faso,(12),M-43 120mm,Mortar,(2011),2011,12,Probably second-hand,NaN,8
1,Algeria,Angola,4,N-262 Fregate,Transport aircraft,1980,1980,4,Second-hand; incl for MP,NaN,8
2,Algeria,Nigeria,(2),Il-28,Bomber aircraft,1967,1968,(2),Second-hand,NaN,8
3,Algeria,Western Sahara,(3),BTR-60PB,APC,(2016),2016,(3),Second-hand; aid,NaN,8
4,Angola,Congo,1,Noratlas,Transport aircraft,1979,1979,1,Second-hand; aid,NaN,8


- When we create the matrix, we add a column called "comment_2" because due to tabulations in the .docx we have several rows displaced one cell to the right.

   We detected that these rows have between 4 and 7 cells with values.

In [4]:
indexes = list(df2[(df2['count_values'].isin([4,5,6,7])) & (df2['number_ordered'].isnull())].index)
for i in indexes:
        df2.loc[i, 'number_ordered']     = df2.loc[i, 'weapon_designation']
        df2.loc[i, 'weapon_designation'] = df2.loc[i, 'weapon_description']
        df2.loc[i, 'weapon_description'] = df2.loc[i, 'year_of_order']
        df2.loc[i, 'year_of_order']      = df2.loc[i, 'year_of_deliveries']
        df2.loc[i, 'year_of_deliveries'] = df2.loc[i, 'number_delivered']
        df2.loc[i, 'number_delivered']   = df2.loc[i, 'comments']
        df2.loc[i, 'comments']           = df2.loc[i, 'comments2']
        
df2 = df2.drop(['comments2'], axis = 1)
df2 = df2.drop(['count_values'], axis = 1)

# we replace parentheses
df2['number_ordered']     = df2['number_ordered'].str.replace('(', '')
df2['number_ordered']     = df2['number_ordered'].str.replace(')', '')
df2['year_of_order']      = df2['year_of_order'].str.replace('(', '')
df2['year_of_order']      = df2['year_of_order'].str.replace(')', '')
df2['year_of_deliveries'] = df2['year_of_deliveries'].str.replace('(', '')
df2['year_of_deliveries'] = df2['year_of_deliveries'].str.replace(')', '')
df2['number_delivered']   = df2['number_delivered'].str.replace('(', '')
df2['number_delivered']   = df2['number_delivered'].str.replace(')', '')

- On the other hand, there are the years of order and the years of delivery of weapons. In the case of delivery years, we found that an order was delivered over several years. That is to say, an order made in 2010 was delivered between 2011-2015. Therefore, we will distribute the number of weapons delivered between the number of years of delivery.

In [5]:
# We count the length of the delivery years. 
# If it is 4, the delivery year is unique. 
# If it is 9, there is a range of delivery years.
df2['len_years'] = df2['year_of_deliveries'].str.len()

# we separate the rows that contain the years of multiple delivery
df3 = df2[df2['len_years'] == 9.0]
df3 = df3.reset_index(drop=True)

# we delete those rows from the df2 because at the end we will append the transformed rows
df2 = df2.drop(df2[df2.len_years == 9.0].index)
df2 = df2.reset_index(drop=True)

df3['start_year_deliveries'] = df3["year_of_deliveries"].str.split("-", n = 1, expand = True)[0]
df3['end_year_deliveries']   = df3["year_of_deliveries"].str.split("-", n = 1, expand = True)[1]
df3['how_many_years']        = pd.to_numeric(df3['end_year_deliveries'], errors='coerce')\
                             - pd.to_numeric(df3['start_year_deliveries'], errors='coerce') + 1

df3['%'] = pd.to_numeric(df3['number_delivered'], errors='coerce')\
          /pd.to_numeric(df3['how_many_years'], errors='coerce')

- There are 3 scenarios for deliveries in multiple years

   The first is that the number of weapons to be distributed is less than or equal to the number of years.
   When that happens, year_of_deliveries = end_year_deliveries

   The second is that the number of weapons / number of years of delivery results in a whole value. In this way, we distribute    the delivery in a uniform manner.

   The third case is that the quotient is not exact. At the moment we distribute as in the second case although we work with      decimals. This could be a section for improvement.

In [6]:
indexes = range(0, len(df3))
for i in indexes:
    if df3['%'][i] <= 1:
        
        df3['year_of_deliveries'][i] = df3['end_year_deliveries'][i]
        df3 = df3.reset_index(drop=True)
        
    else:
        
        row      = [df3[i:i+1]]
        how_many = df3['how_many_years'][i]-1
        
        df3['number_delivered'][i]   = df3['%'][i]
        df3['year_of_deliveries'][i] = df3['start_year_deliveries'][i]
        df3 = df3.append(pd.concat(row * how_many))  
        df3 = df3.reset_index(drop=True)
        
        
for i in range(max(indexes), len(df3)):
    if ((df3.loc[i,'country_supplier']       == df3.loc[i-1,'country_supplier'])
        and (df3.loc[i,'country_recipient']  == df3.loc[i-1,'country_recipient'])
        and (df3.loc[i,'number_ordered']     == df3.loc[i-1,'number_ordered'])
        and (df3.loc[i,'weapon_designation'] == df3.loc[i-1,'weapon_designation'])
        and (df3.loc[i,'weapon_description'] == df3.loc[i-1,'weapon_description'])
        and (df3.loc[i,'year_of_order']      == df3.loc[i-1,'year_of_order'])
        and (df3.loc[i,'%']                  == df3.loc[i-1,'%'])):
        df3['year_of_deliveries'][i] = int(df3['year_of_deliveries'][i - 1]) + 1
    else:
        df3['year_of_deliveries'][i] = int(df3['start_year_deliveries'][i]) + 1

- Now that we have divided the multiple delivery years we can join the df2 with the df3. For them we delete the columns that we have been adding to make the transformations

In [7]:
df2    = df2.drop(['len_years'], axis = 1)
df3    = df3.drop(['len_years', 'start_year_deliveries', 'end_year_deliveries', 'how_many_years', '%'], axis = 1)
frames = [df2, df3]
df4    = pd.concat(frames)
df4    = df4.reset_index(drop=True)

In [8]:
# Finally, we see that there are several orders for which we do not have information on the quantity ordered / delivered. 
# These rows represent 1% of the sample, so we delete them.
df4 = df4.drop(df4[(df4['number_ordered'].isnull()) & (df4['number_delivered'].isnull())].index)
df4 = df4.reset_index(drop = True)

# We see what is wrong in the column number delivered. If there is missing information or there is displaced text
df4['aux'] = np.nan
for i in range(0,len(df4)):
    try:
        df4['aux'][i] = int(df4['number_delivered'][i])
    except:
        df4['aux'][i] = 'string'
        
df4 = df4.drop(df4[df4['aux'] == 'string'].index)
df4 = df4.drop(['aux'], axis = 1)
df4['number_delivered'] = df4['number_delivered'].astype(int)

8. Save dataframe as .csv

In [9]:
df4.to_csv('05_Output_Weapons.csv', header = True, sep = ';', encoding = 'utf8', index = False)